# 네이버 뉴스 기사 크롤링

언론사:연합뉴스,연합인포맥스,이데일리

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import datetime

error_date=[]

#본문내용크롤링
def get_news(url):  
    
    header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'}
    html = requests.get(url, headers=header)
    bsoup = BeautifulSoup(html.content, 'html.parser')
    

    # 기사 제목
    title = bsoup.select('h3#articleTitle')[0].text

   
    # 날짜 
    pdate = bsoup.select('.t11')[0].get_text()[:10]
      

    # 기사 본문
    content = bsoup.select('#articleBodyContents')[0].get_text().replace('\n', " ")
    ctext = content.replace("// flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}", "") 
   
    return title, pdate, ctext

#네이버 접속
def naver_news_crawling(query, start_date, end_date, s_from, e_to, page):
    
    title_list=[]
    date_list=[]
    contents_list=[]
    
    #최대5페이지까지
    while page<50:
  
        #print(page)
        url="https://search.naver.com/search.naver?&where=news&query="+query+"&sm=tab_pge&sort=2&photo=0&field=0&reporter_article=&pd=3&ds="+start_date+"&de="+ end_date +"&docid=&nso=so:da,p:from"+s_from+"to"+e_to+",a:all&mynews=1&start="+str(page)+"&refresh_start=0"
        #print(url)
        
        #header 추가 
        header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36',
                  'cookie':'news_office_checked=1001,1018,2227'}
    
        req = requests.get(url,headers=header)
        #print(url)
        cont = req.text
        soup = BeautifulSoup(cont, 'html.parser')

        for urls in soup.select("a.info"):
            try :
                if urls["href"].startswith("https://news.naver.com"):
                   
                    title_text, date_text, contents_text = get_news(urls["href"])
                    
                    title_list.append(title_text)
                    date_list.append(date_text)
                    contents_list.append(contents_text)
            except Exception as e:
                print(e) 
                continue
        page += 10
        
    result={}   
    result= {'date' : date_list, 'title':title_list,'contents': contents_list}
    
    df=[]
    df=pd.DataFrame(result)
    
    return df

#초기값 설정
page = 1
query = "금리" 
date = datetime.date(2009,1,1) #시작날짜
start_date=date.strftime('%Y.%m.%d')
end_date=start_date
s_from = start_date.replace(".","")
e_to = start_date.replace(".","")
standard_date=start_date
#----------------------------------------------------------

#---------------메인으로 돌아가는 부분----------------------
while(standard_date!="2010.01.01"):#끝날짜 
    try:
        
        print("---------------"+standard_date+"날짜의 크롤링을 시작합니다---------------")
    
        new_df = naver_news_crawling(query, start_date, end_date, s_from, e_to, page)
        new_df=new_df.drop_duplicates()
        new_df['title'] = new_df['title'].str.encode('utf-8', 'ignore').str.decode('utf-8')
        new_df['contents'] = new_df['contents'].str.encode('utf-8', 'ignore').str.decode('utf-8')
        new_df.to_csv(r"C:\Users\HanBi\Desktop\2021\project1\crawling\금리2009.csv",encoding='utf-8',index=False, header=False, mode='a')
    
        date += datetime.timedelta(days=1)
        start_date=date.strftime('%Y.%m.%d')
        end_date=start_date
    
        s_from = start_date.replace(".","")
        e_to = start_date.replace(".","")
    
        standard_date=start_date
        print("성공")
    except Exception as e:
        error_date.append(standard_date)
        print(e)
        print("실패")
        continue
    
    
print("---------------크롤링을 종료합니다---------------")